<a href="https://colab.research.google.com/github/singhsa3/Emotions/blob/main/librosaModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# PyTorch libraries and modules
import torch
from torch.autograd import Variable
from torch.nn import GroupNorm, Linear, ReLU, GELU, CrossEntropyLoss, Sequential, Conv2d, Conv1d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout,AvgPool2d
from torch.optim import Adam, SGD
import numpy as np
import torch.nn as nn
import torch.nn.functional as F


In [2]:
from google.colab import drive
drive.mount('/content/drive')
pathG='/content/drive/MyDrive/Pract/data'
cp = '/content/drive/MyDrive/Pretrained/wav2vec_small.pt'
sample="/content/drive/MyDrive/Pract/data/voice_samples/1173_GM1001_1326493712.wav"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pickle
with open(pathG+"/featuresdict.pickle", "rb") as f:
  features=pickle.load(f)
with open(pathG+"/filenameslist.pickle", "rb") as f:
  fnames=pickle.load(f)
import pandas as pd
df=pd.read_csv(pathG+"/labels/Yared Alemu_fear.csv")
df = df[(df['name'].isin (features.keys()))].reset_index()
df.head()

,index,Unnamed: 0,therapist,name,emotion_type,rating,emotion
0,0,8265,Yared Alemu,6529_53113_3605013943.wav,fear,high,1
1,1,5540,Yared Alemu,8953_39117_2192499364.wav,fear,low,0
2,2,6675,Yared Alemu,1940_39117_1596672000.wav,fear,low,0
3,3,4640,Yared Alemu,53490_53113_1571011200.wav,fear,low,0
4,4,5525,Yared Alemu,8953_39117_4097508814.wav,fear,low,0


In [4]:
x=features[df.iloc[0][3]]
x= np.pad(x, ((10,10),(10, 6500-x.shape[1])), 'constant')
arr =np.expand_dims(x, axis=0)
#print(arr.shape)
for i,row in df.iterrows():
  if i>0:
    ft = features[row['name']]
    ft= np.pad(ft, ((10,10),(10, 6500-ft.shape[1])), 'constant')
    arr2 = np.expand_dims(ft, axis=0)
    #print(arr2.shape)
    arr = np.vstack((arr,arr2))
labels=np.array(df.emotion)

In [5]:
import pickle
with open(pathG+"/curfile.pickle", "wb") as f:
  pickle.dump((arr, labels),f)

In [1]:
# PyTorch libraries and modules
import torch
from torch.autograd import Variable
from torch.nn import GroupNorm, Linear, ReLU, GELU, CrossEntropyLoss, Sequential, Conv2d, Conv1d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout,AvgPool2d
from torch.optim import Adam, SGD
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
class Fp32GroupNorm(nn.GroupNorm):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def forward(self, input):
        output = F.group_norm(
            input.float(),
            self.num_groups,
            self.weight.float() if self.weight is not None else None,
            self.bias.float() if self.bias is not None else None,
            self.eps,
        )
        return output.type_as(input)


In [2]:
from google.colab import drive
drive.mount('/content/drive')
pathG='/content/drive/MyDrive/Pract/data'
cp = '/content/drive/MyDrive/Pretrained/wav2vec_small.pt'
sample="/content/drive/MyDrive/Pract/data/voice_samples/1173_GM1001_1326493712.wav"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pickle
with open(pathG+"/curfile.pickle", "rb") as f:
  abc = pickle.load(f)
arr=abc[0]
labels=abc[1]

In [4]:
from sklearn.model_selection import train_test_split
train_x, val_x, train_y, val_y = train_test_split(arr, labels, test_size = 0.3)

In [5]:
# converting training images into torch format
train_x = train_x.reshape(train_x.shape[0], 1, train_x.shape[1], train_x.shape[2])
train_x  = torch.from_numpy(train_x)

# converting the target into torch format
train_y = train_y.astype(int);
train_y = torch.from_numpy(train_y)

# shape of training data
train_x.shape, train_y.shape
# converting validation images into torch format
val_x = val_x.reshape(val_x.shape[0], 1, val_x.shape[1], val_x.shape[2])
val_x  = torch.from_numpy(val_x)

# converting the target into torch format
val_y = val_y.astype(int);
val_y = torch.from_numpy(val_y)

# shape of validation data
val_x.shape, val_y.shape

(torch.Size([127, 1, 148, 6510]), torch.Size([127]))

In [6]:
import torch
torch.cuda.empty_cache()
!nvidia-smi
a = torch.cuda.memory_allocated(0)
c = torch.cuda.memory_reserved(0)
print(a,c)

Sun Jun  5 01:33:13 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [7]:
import torch
from torch.utils.data import DataLoader, TensorDataset

In [8]:
class Net(Module):   
    def __init__(self):
        super(Net, self).__init__()

        self.cnn_layers = Sequential(
            # Defining a 2D convolution layer
            Conv2d(1, 4, kernel_size=2, stride=1, padding=0),
            BatchNorm2d(4),
            ReLU(inplace=True),
            AvgPool2d(kernel_size=4, stride=1),
            # Defining another 2D convolution layer
            Conv2d(4, 16, kernel_size=2, stride=1, padding=0),
            BatchNorm2d(16),
            ReLU(inplace=True),
            AvgPool2d(kernel_size=4, stride=1),
            # Defining another 2D convolution layer
            Conv2d(16, 64, kernel_size=2, stride=1, padding=0),
            BatchNorm2d(64),
            ReLU(inplace=True),
            AvgPool2d(kernel_size=4, stride=1),
            
        )

        self.linear_layers = Sequential(
            Linear(56558592, 2)
        )

    # Defining the forward pass    
    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x

In [9]:
# defining the model
model = Net()
# defining the optimizer
optimizer = Adam(model.parameters(), lr=0.07)
# defining the loss function
criterion = CrossEntropyLoss()
# checking if GPU is available
if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()
    
#print(model)
# Define the batch size and the number of epochs
BATCH_SIZE = 5
N_EPOCHS = 50
torch.cuda.empty_cache()
# Use torch.utils.data to create a DataLoader 
# that will take care of creating batches 
dataset = TensorDataset(train_x, train_y)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

# Get the dataset size for printing (it is equal to N_SAMPLES)
dataset_size = len(dataloader.dataset)
# empty list to store training losses
train_losses = [100,100,100,100,100]
# empty list to store validation losses
val_losses = []

# Loop over epochs
for epoch in range(N_EPOCHS):
    mnl = np.mean(train_losses)
    print(f"Epoch {epoch + 1}\n-------------------------------")

    # Loop over batches in an epoch using DataLoader
    for id_batch, (x_batch, y_batch) in enumerate(dataloader):
      if torch.cuda.is_available():
        x_batch = x_batch.cuda()
        y_batch = y_batch.cuda()
        #x_val = val_x.cuda()
        #y_val = val_y.cuda()

        y_batch_pred = model(x_batch)
        #output_val = model(x_val)

        loss = criterion(y_batch_pred, y_batch)
        #loss_val = criterion(output_val, y_val)
        #train_losses.append(loss)
        #val_losses.append(loss_val)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Every 100 batches, print the loss for this batch
        # as well as the number of examples processed so far 
        if id_batch % 100 == 0:
            loss, current = loss.item(), (id_batch + 1)* len(x_batch)
            print(f"loss: {loss:>7f}  [{current:>5d}/{dataset_size:>5d}]")    
    train_losses.pop()
    train_losses.insert(0,loss.detach().cpu().numpy())
    #if np.mean(train_losses)>mnl*1.10:
      #break
torch.cuda.empty_cache()

Epoch 1
-------------------------------
loss: 0.695104  [    5/  295]
Epoch 2
-------------------------------
loss: 8.908305  [    5/  295]
Epoch 3
-------------------------------
loss: 0.404806  [    5/  295]
Epoch 4
-------------------------------
loss: 0.520656  [    5/  295]
Epoch 5
-------------------------------
loss: 0.550291  [    5/  295]
Epoch 6
-------------------------------
loss: 0.660351  [    5/  295]
Epoch 7
-------------------------------
loss: 2.992746  [    5/  295]
Epoch 8
-------------------------------
loss: 0.737954  [    5/  295]
Epoch 9
-------------------------------
loss: 0.624739  [    5/  295]
Epoch 10
-------------------------------
loss: 0.680556  [    5/  295]
Epoch 11
-------------------------------
loss: 0.701996  [    5/  295]
Epoch 12
-------------------------------
loss: 0.695752  [    5/  295]
Epoch 13
-------------------------------
loss: 0.560792  [    5/  295]
Epoch 14
-------------------------------
loss: 0.716229  [    5/  295]
Epoch 15
------

In [10]:
torch.save(model, 'model.bin')

In [11]:
model = torch.load('model.bin')
model.eval()

Net(
  (cnn_layers): Sequential(
    (0): Conv2d(1, 4, kernel_size=(2, 2), stride=(1, 1))
    (1): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): AvgPool2d(kernel_size=4, stride=1, padding=0)
    (4): Conv2d(4, 16, kernel_size=(2, 2), stride=(1, 1))
    (5): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): AvgPool2d(kernel_size=4, stride=1, padding=0)
    (8): Conv2d(16, 64, kernel_size=(2, 2), stride=(1, 1))
    (9): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): AvgPool2d(kernel_size=4, stride=1, padding=0)
  )
  (linear_layers): Sequential(
    (0): Linear(in_features=56558592, out_features=2, bias=True)
  )
)

In [12]:
import torch
torch.cuda.empty_cache()
!nvidia-smi
a = torch.cuda.memory_allocated(0)
c = torch.cuda.memory_reserved(0)
print(a,c)

Sun Jun  5 02:17:33 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P0    46W / 250W |   3361MiB / 16280MiB |      1%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [13]:
from sklearn.metrics import accuracy_score
# prediction for validation set
predictions=[]
dataset = TensorDataset(val_x, val_y)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)
for id_batch, (x_batch, y_batch) in enumerate(dataloader):
      if torch.cuda.is_available():
        x_batch = x_batch.cuda()
        y_batch = y_batch.cuda()
        x_val = val_x.cuda()
        y_val = val_y.cuda()
        output = model(x_batch)
        softmax = torch.exp(output).cpu()
        prob = list(softmax.detach().cpu().numpy())
        prediction = np.argmax(prob, axis=1)
        predictions = predictions+list(prediction)

        

# accuracy on validation set
accuracy_score(val_y, np.array(predictions))

0.47244094488188976